In [3]:
# download the required packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
print("Ready!")

Ready!


In [11]:
# Convert Sentinel-2 scene to hdf5 format
import h5py

# Define the input Sentinel-2 image file path
input_image_file = r"C:\Users\lilly\Documents\Sentinel downloads\Santa Barbara\2021\August 21\S2B_MSIL2A_20210831T173859_N0301_R098_T14VNM_20210831T214202.SAFE\MTD_MSIL2A.xml"

# Define the output HDF5 file path
output_hdf5_file = r"C:\Users\lilly\Documents\Sentinel_downloads\SantaBarbara\Aug21.h5" 

In [8]:
# Open the Sentinel-2 image using rasterio
with rasterio.open(input_image_file) as src:
    # Read the image bands
    bands = src.read()
    num_bands, height, width = bands.shape

    # Create a new HDF5 file
    with h5py.File(output_hdf5_file, 'w') as f:
        # Create a group for the bands
        bands_group = f.create_group('bands')

        # Create a dataset for each band
        for band_idx in range(num_bands):
            band_name = f'band_{band_idx + 1}'
            band_data = bands[band_idx, :, :]

            bands_group.create_dataset(band_name, data=band_data)

        # Add metadata to the HDF5 file
        f.attrs['num_bands'] = num_bands
        f.attrs['height'] = height
        f.attrs['width'] = width

# Print a success message
print('Sentinel-2 image converted to HDF5 format successfully.')

RasterioIOError: C:/Users/lilly/Documents/Sentinel downloads/Santa Barbara/2021/August 21/S2B_MSIL2A_20210831T173859_N0301_R098_T14VNM_20210831T214202.SAFE: Permission denied

In [9]:
# Set the data path where data will be downloaded (modify as needed!)
site = 'SERC'
year = '2021'
data_root_path = f'./data/{site}/{year}/'
h5_path = os.path.join(data_root_path, 'refl')

# create the data directory if it doesn't exist
os.makedirs(h5_path, exist_ok=True)

# print the h5 path
print(data_root_path)

# display the contents in the data directory
print(os.listdir(h5_path))

# set the variable h5_file to be the full path of the downloaded reflectance h5 tile
h5_file = os.path.join(h5_path, 'filename.h5')

# read in the reflectance data using the aop_h5refl2array function
def aop_h5refl2array(h5_file, refl_band):
    # implementation of aop_h5refl2array function
    pass

refl, refl_metadata, wavelengths = aop_h5refl2array(h5_file, 'Reflectance')

# plot a single band of data
band56 = refl[:, :, 55]  # note Python uses 0-based indexing, so 55 corresponds to band 56
plt.imshow(band56 / refl_metadata['scale_factor'], extent=refl_metadata['extent'], cmap='gist_earth',
           vmin=0, vmax=0.3)
plt.title('SERC Tile Band 56')
plt.colorbar(label='Reflectance')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

# pull out the true-color and false-color band combinations
rgb_bands = (58, 34, 19)  # set the red, green, and blue bands
cir_bands = (90, 34, 19)  # set the color infrared bands

# stack the 3-band combinations (rgb and cir) using stack_rgb function
def stack_rgb(refl, bands):
    # implementation of stack_rgb function
    pass

rgb_unscaled = stack_rgb(refl, rgb_bands)
cir_unscaled = stack_rgb(refl, cir_bands)

# apply the reflectance scale factor
rgb = rgb_unscaled / refl_metadata['scale_factor']
cir = cir_unscaled / refl_metadata['scale_factor']

./data/SERC/2021/
[]


TypeError: cannot unpack non-iterable NoneType object

In [10]:
# plot the true color image (rgb)
plt.imshow(rgb, extent=refl_metadata['extent'])
plt.title('SERC RGB Image')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

# plot the false color image (color infrared / cir)
plt.imshow(cir, extent=refl_metadata['extent'])
plt.title('SERC Color Infrared Image')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

# read in the spectral information from the center pixel (of the 1000 x 1000 m tile) to a dataframe
serc_pixel_df = pd.DataFrame()
serc_pixel_df['reflectance'] = refl[500, 500, :]
serc_pixel_df['wavelengths'] = wavelengths

# plot the spectral signature
plt.plot(serc_pixel_df['wavelengths'], serc_pixel_df['reflectance'], linewidth=2)
plt.title('Spectral Signature for SERC Center Pixel')
plt.xlabel('Wavelength, nm')
plt.ylabel('Reflectance')
plt.xlim([np.min(serc_pixel_df['wavelengths']), np.percentile(serc_pixel_df['wavelengths'], 98)])
plt.ylim([np.min(serc_pixel_df['reflectance']), np.percentile(serc_pixel_df['reflectance'], 98)])
plt.grid(True)
plt.show()

NameError: name 'rgb' is not defined